# ***MAIN FUNCTIONALITY***

In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns

from meteo.Site import Site
from solar.SolarPVPanel import SolarPVPanel
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging
from misc.util import load_pv_model

configure_logging()
pd.options.display.float_format = '{:.3f}'.format

# Set up site location and get TMY data
name = ""
address = "York St, Belfast, BT15 1ED"
client = "Ulster University"

site = Site(name=name, 
            address=address, 
            client=client, 
            latitude=54.60452, 
            longitude=-5.92860, 
            size=100)

# Set up PV panel being used
panel_kwp = 0.3538
size_m2 = 1.990
eff = 0.2237
cell_temp_coeff = -0.004
cell_NOCT = 48
lifespan = 25
pv_eol_derating = 0.88
refraction_index = 0.05

pv_panel = SolarPVPanel(panel_kwp, size_m2, eff, cell_temp_coeff, 
                        cell_NOCT, lifespan, pv_eol_derating, refraction_index)


# Set up PV arrays
num_pv_panels = 4
surface_pitch = 35
surface_azimuth = -90
albedo = 0.2
cost_per_kWp = 1250

pv_array_1 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, surface_azimuth, albedo, cost_per_kWp)
pv_array_2 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, -45)
pv_array_3 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 0)
pv_array_4 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 45)
pv_array_5 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 90)

pv_arrays = [pv_array_1, pv_array_2, pv_array_3, pv_array_4, pv_array_5]


# Run the model
pv_model = SolarPVModel(site, pv_arrays)

# Save model as .wmm file
pv_model.save_model()

# Load .wwm model from file
pv_model = load_pv_model()

print("")
print(f"{sum(array.pv_kwp for array in pv_model.arrays)}kWp Solar PV modelled covering a {sum(array.area_m2 for array in pv_model.arrays)}m2 area.")


2024-02-22 19:32:12,774 - INFO - Fetching TMY data for latitude: 54.60452, longitude: -5.9286
2024-02-22 19:32:13,771 - INFO - HTTP Request: GET https://re.jrc.ec.europa.eu/api/tmy?lat=54.60452&lon=-5.9286&startyear=2005&endyear=2015&outputformat=json "HTTP/1.1 200 OK"
2024-02-22 19:32:14,123 - INFO - TMY data obtained for: 54.60452, longitude: -5.9286
2024-02-22 19:32:14,124 - INFO - *******************
2024-02-22 19:32:14,124 - INFO - Solar PV Panel created: Size: 0.3538kW, Size: 1.99m2, Efficiency: 0.2237%, Lifespan: 25yrs
2024-02-22 19:32:14,126 - INFO - Solar PV array created: Size: 1.415kW, Size: 7.96m2,Azimuth: -90deg, Lifespan: 25yrs,Pitch: 35deg, Efficiency: 0.2237%
2024-02-22 19:32:14,126 - INFO - Solar PV array created: Size: 1.415kW, Size: 7.96m2,Azimuth: -45deg, Lifespan: 25yrs,Pitch: 35deg, Efficiency: 0.2237%
2024-02-22 19:32:14,127 - INFO - Solar PV array created: Size: 1.415kW, Size: 7.96m2,Azimuth: 0deg, Lifespan: 25yrs,Pitch: 35deg, Efficiency: 0.2237%
2024-02-22 19:


7.075kWp Solar PV modelled covering a 39.8m2 area.


In [2]:
pv_model.summary

PV_Gen_kWh_Annual              6322.910
PV_Gen_kWh_Lifetime          148209.017
E_POA_kWm2_Annual               956.222
Panel_POA_kWm2_Annual           944.607
IAM_Loss_kWh_Annual              77.775
PV_Thermal_Loss_kWh_Annual       51.480
Low_Light_Loss_kWh_Annual       293.198
E_Beam_kWm2_Annual              421.631
E_Diffuse_kWm2_Annual           517.524
E_Ground_kWm2_Annual             17.067
ET_HRad_kWm2_Annual            2270.579
Array_Temp_C_Avg                 11.311
Ambient_Temperature_C_Avg         8.899
dtype: float64

In [4]:
print(pv_model.models[0]["model_result"].E_POA_kWm2.sum())
print(pv_model.models[1]["model_result"].E_POA_kWm2.sum())
print(pv_model.models[2]["model_result"].E_POA_kWm2.sum())
print(pv_model.models[3]["model_result"].E_POA_kWm2.sum())
print(pv_model.models[4]["model_result"].E_POA_kWm2.sum())

869.1227452409016
989.4152531988715
1041.0328100686736
1000.7969560619578
880.7426043828083


In [5]:
pv_model.models[0]["model_result"].isna().sum()

Hour_of_Day              0
Day_of_Year              0
Week_of_Year             0
Month_of_Year            0
Wind_Speed_ms            0
Ambient_Temperature_C    0
Declination_Angle        0
Solar_Time               0
Hour_Angle               0
AOI                      0
Zenith_Angle             0
E_Beam_kWm2              0
E_Diffuse_kWm2           0
E_Ground_kWm2            0
E_POA_kWm2               0
Panel_POA_kWm2           0
IAM_Loss_kWh             0
ET_HRad_kWm2             0
PV_Derated_Eff           0
Array_Temp_C             0
PV_Gen_kWh               0
PV_Thermal_Loss_kWh      0
Low_Light_Loss_kWh       0
dtype: int64

In [6]:
pv_model.models[1]["model_result"].sum()

Hour_of_Day              100740.000
Day_of_Year             1603080.000
Week_of_Year             232752.000
Month_of_Year             57168.000
Wind_Speed_ms             53080.970
Ambient_Temperature_C     77957.930
Declination_Angle             0.000
Solar_Time               101660.207
Hour_Angle               -51896.890
AOI                      788401.501
Zenith_Angle             788400.000
E_Beam_kWm2                 454.824
E_Diffuse_kWm2              517.524
E_Ground_kWm2                17.067
E_POA_kWm2                  989.415
Panel_POA_kWm2              978.305
IAM_Loss_kWh                 14.933
ET_HRad_kWm2               2270.579
PV_Derated_Eff             8738.978
Array_Temp_C              99822.627
PV_Gen_kWh                 1312.744
PV_Thermal_Loss_kWh          10.845
Low_Light_Loss_kWh           57.497
dtype: float64

# ***Temperature Tests***

In [7]:
import math

def calc_array_temp_homer(
    e_poa,
    ambient_temp,
    cell_temp_coeff=-0.0035,
    electrical_eff=0.21,
    cell_NOCT=42,
    ambient_NOCT=20,
    e_poa_NOCT=800,
    cell_temp_STC=25,
    transmittance_absorptance=0.9,
):
    """Calculates the cell temperature of a PV panel.

    Parameters:
    - e_poa: Plane of array irradiance in kW/m^2.
    - ambient_temp: Ambient temperature in degrees Celsius.
    - cell_temp_coeff: Temperature coefficient of the PV cell.
    - electrical_eff: Electrical efficiency of the PV panel.
    - cell_NOCT, ambient_NOCT: Nominal operating cell temperature and the corresponding ambient temperature.
    - e_poa_NOCT: Irradiance at NOCT conditions in W/m^2.
    - cell_temp_STC: Cell temperature at standard test conditions in degrees Celsius.
    - transmittance_absorptance: Transmittance and absorptance product of the PV panel.

    Returns:
    - Cell temperature of the PV panel.
    """
    temp_factor = (cell_NOCT - ambient_NOCT) * ((e_poa * 1000) / e_poa_NOCT)
    numerator = ambient_temp + temp_factor * (
        1
        - (electrical_eff * (1 - cell_temp_coeff * cell_temp_STC))
        / transmittance_absorptance
    )
    denominator = 1 + temp_factor * (
        cell_temp_coeff * electrical_eff / transmittance_absorptance
    )

    return numerator / denominator


def calc_array_temp_pvsyst(e_poa, ambient_temp, windspeed, uc=29, uv=1.2):
    u = uc + (uv * windspeed)
    tcell = ambient_temp + (1/u) * (0.9 * e_poa * 1000 * (1-0.21))
    return tcell

def calc_array_temp_sandia(e_poa: float, ambient_temp: float, wind_speed: float, 
                           a: float = -3.47, b: float = -0.0594) -> float:
    """
    Calculate the temperature of a photovoltaic (PV) array based on the Sandia method.
    
    Parameters:
    - e_poa (float): Plane of array irradiance in kW/m^2. Represents the solar irradiance incident on the PV array.
    - ambient_temp (float): Ambient temperature in degrees Celsius.
    - wind_speed (float): Wind speed in m/s at the site of the PV array.
    - a (float): Coefficient a in the exponential model, defaulting to -3.47.
    - b (float): Coefficient b in the exponential model, defaulting to -0.0594.
    
    Returns:
    - float: Estimated temperature of the PV array in degrees Celsius.
    """
    array_temp = e_poa * 1000 * math.exp(a + b * wind_speed) + ambient_temp
    return array_temp

def calc_array_temp_faiman(e_poa, ambient_temp, wind_speed, U_0=25, U_1=6.84):
    array_temp = ambient_temp + (e_poa * 1000 / (U_0 + U_1 * wind_speed))
    return array_temp


# Example usage:
e_poa=0.5
ambient_temp=25
wind_speed=1

list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []


print("")
wind_speed=10
for i in range(1):
    cell_temperature = calc_array_temp_homer(e_poa, ambient_temp)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Homer model with {wind_speed}m/s windspeed")
    list_2.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(21):
    cell_temperature = calc_array_temp_pvsyst(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the PVSyst model with {wind_speed}m/s windspeed")
    list_3.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(21):
    cell_temperature = calc_array_temp_sandia(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Sandia model with {wind_speed}m/s windspeed")
    list_4.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(21):
    cell_temperature = calc_array_temp_faiman(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Faiman model with {wind_speed}m/s windspeed")
    list_5.append(cell_temperature)
    wind_speed += 1



The cell temperature is 35.66°C from the Homer model with 10m/s windspeed

The cell temperature is 37.26°C from the PVSyst model with 0m/s windspeed
The cell temperature is 36.77°C from the PVSyst model with 1m/s windspeed
The cell temperature is 36.32°C from the PVSyst model with 2m/s windspeed
The cell temperature is 35.90°C from the PVSyst model with 3m/s windspeed
The cell temperature is 35.52°C from the PVSyst model with 4m/s windspeed
The cell temperature is 35.16°C from the PVSyst model with 5m/s windspeed
The cell temperature is 34.82°C from the PVSyst model with 6m/s windspeed
The cell temperature is 34.51°C from the PVSyst model with 7m/s windspeed
The cell temperature is 34.21°C from the PVSyst model with 8m/s windspeed
The cell temperature is 33.93°C from the PVSyst model with 9m/s windspeed
The cell temperature is 33.67°C from the PVSyst model with 10m/s windspeed
The cell temperature is 33.42°C from the PVSyst model with 11m/s windspeed
The cell temperature is 33.19°C fr

In [8]:
%%timeit
cell_temperature = calc_array_temp_faiman(e_poa, ambient_temp, wind_speed)

In [ ]:
%%timeit
cell_temperature = calc_array_temp_sandia(e_poa, ambient_temp, wind_speed)

In [ ]:
%%timeit
cell_temperature = calc_array_temp_pvsyst(e_poa, ambient_temp, wind_speed)

In [ ]:
%%timeit
cell_temperature = calc_array_temp_homer(e_poa, ambient_temp)

# Low Light Losses Visualisation Test

In [ ]:
import numpy as np
import plotly.express as px

# Define the modified logistic function
def calc_low_light_losses(pv_kwp, e_poa, k=0.0075, midpoint=25):
    """
    Modified logistic function to calculate efficiency based on irradiance,
    with a minimum efficiency level.

    Parameters:
    pv_kwp (float): The rated solar PV size (kWp).
    e_poa (float): The irradiance incident on array (W/m2).
    k (float): The steepness of the curve.
    midpoint (float): The irradiance at which the efficiency is at its midpoint.

    Returns:
    float: The calculated efficiency at the given irradiance.
    """
    pv_kwp_min = pv_kwp * 0.6
    eff = pv_kwp_min + (pv_kwp - pv_kwp_min) / (1 + np.exp(-k * (e_poa - midpoint)))
    return eff


# Generate a range of irradiance values from 0 to 1000 W/m2 and set L to 1 kWp
L = 1
irradiance_range = np.linspace(0, 1000, 1001)
efficiencies = calc_low_light_losses(L, irradiance_range)

# Plot Graph of results
fig = px.line(efficiencies[:], y=efficiencies[:], x = irradiance_range[:])

fig.update_layout(xaxis_title="Irradiation (W/m2)", yaxis_title="kWp Output",
                  title = "Line graph showing PV kWp output due to low irradiance losses")

In [ ]:
calc_low_light_losses(1, 200)

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", 
                                         "PV_Thermal_Loss_kWh_Total", "Low_Light_Loss_kWh_Total"]])

In [ ]:
pv_model.summary_grouped.hourly.columns

# ***Performance Test***

In [ ]:
# Performance Testing
# import pstats
# import cProfile
# from meteo.Site import Site
# from solar.SolarPVPanel import SolarPVPanel
# from solar.SolarPVArray import SolarPVArray
# from solar.SolarPVModel import SolarPVModel
# from misc.log_config import configure_logging
# from misc.util import load_pv_model
# configure_logging()
# cProfile.run("SolarPVModel(site=Site(latitude=54.60452, longitude=-5.92860, size=100), arrays=[SolarPVArray(SolarPVPanel(), 4), SolarPVArray(SolarPVPanel(), 4), SolarPVArray(SolarPVPanel(), 4), SolarPVArray(SolarPVPanel(), 4), SolarPVArray(SolarPVPanel(), 4)])", 'performance_test.prof')
# p = pstats.Stats('performance_test.prof')
# p.sort_stats('tottime').print_stats()
# p.sort_stats('cumtime').print_stats()

# ***Example Running Model with variable inputs***

In [ ]:
# %%timeit

import pandas as pd
import plotly.express as px
import seaborn as sns

from meteo.Site import Site
from solar.SolarPVPanel import SolarPVPanel
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging
from misc.util import load_pv_model

configure_logging()
pd.options.display.float_format = '{:.3f}'.format

# Set up site location and get TMY data
name = ""
address = "York St, Belfast, BT15 1ED"
client = "Ulster University"

site = Site(name=name, 
            address=address, 
            client=client, 
            latitude=54.60452, 
            longitude=-5.92860, 
            size=100)

In [ ]:
# %%timeit

# Set up PV panel being used
pv_panel = SolarPVPanel()

# Set up PV arrays
num_pv_panels = 4
surface_pitch = 35
surface_azimuth = -90
albedo = 0.2
cost_per_kWp = 1250

pv_array_1 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, surface_azimuth, albedo, cost_per_kWp)
pv_array_2 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, -45)
pv_array_3 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 0)
pv_array_4 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 45)
pv_array_5 = SolarPVArray(pv_panel, num_pv_panels, surface_pitch, 90)

pv_arrays = [pv_array_1, pv_array_2, pv_array_3, pv_array_4, pv_array_5]

# Run the model
pv_model = SolarPVModel(site, pv_arrays)

print("")
print(f"{sum(array.pv_kwp for array in pv_model.arrays)}kWp Solar PV modelled")


In [ ]:
import plotly.express as px

px.bar(pv_model.summary_grouped.hourly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["E_Beam_kWm2_Avg", "E_Diffuse_kWm2_Avg", "E_Ground_kWm2_Avg"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["Array_Temp_C", "Ambient_Temperature_C"]])

In [ ]:
px.line(pv_model.summary_grouped.monthly[["ET_HRad_kWm2_Avg", "E_POA_kWm2_Avg", "Panel_POA_kWm2_Avg"]])

In [ ]:
pv_model.summary

In [ ]:
pv_model.combined_model

# ***Visualisation***

In [ ]:
import plotly.express as px
import seaborn as sns

In [ ]:
pv_model.array_model(1).head()

In [ ]:
print(f"PV Gen: {round(pv_model.all_models.PV_Gen_kWh_Array_1.sum(), 3)}kWh")
print(f"E POA: {round(pv_model.all_models.E_POA_kWm2_Array_1.sum(), 3)}Whm2")

In [ ]:
px.bar(pv_model.summary_grouped.hourly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.weekly[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.daily[["PV_Gen_kWh_Total", "IAM_Loss_kWh_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["Array_Temp_C", "Ambient_Temperature_C"]])


In [ ]:
sns.lineplot(pv_model.models[0]["model_result"][["Array_Temp_C", "Ambient_Temperature_C"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

In [ ]:
sns.lineplot(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

# ***VISUAL TESTING***

In [ ]:
variables = ['E_Beam_kWm2', 'E_Diffuse_kWm2', 'E_Ground_kWm2', 'E_POA_kWm2', 'ET_HRad_kWm2', 
            'Array_Temp_C', 'PV_Gen_kWh', 'AOI', 'Zenith_Angle']

testy = pv_model.all_models

variable = "PV_Gen_kWh"

variable_1 = f"{variable}_Array_1"
variable_2 = f"{variable}_Array_2"
variable_3 = f"{variable}_Array_3"
variable_4 = f"{variable}_Array_4"
variable_7 = f"{variable}_Total"
variable_8 = f"{variable}_Avg"

if variable_1 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_1].mean())
if variable_2 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_2].mean())
if variable_3 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_3].mean())
if variable_4 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_4].mean())
if variable_7 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_7].mean(), alpha=0.4, ls="--")
if variable_8 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_8].mean(), alpha=0.4, ls="--")

In [ ]:
sns.lineplot(pv_model.combined_model["Array_Temp_C_Avg"], alpha = 0.2, ls = "--", color = "orange")
sns.lineplot(pv_model.combined_model["Ambient_Temperature_C"])

In [ ]:
sns.barplot(pv_model.summary_grouped.hourly.Array_Temp_C_Avg)

In [ ]:
sns.lineplot(pv_model.combined_model.groupby("Hour_of_Day")["PV_Gen_kWh_Total"].mean())

In [ ]:
pv_model.all_models["PV_Gen_kWh_Total"].sum()

In [ ]:
pv_model.all_models.columns

In [ ]:
sns.lineplot(pv_model.all_models["PV_Thermal_Loss_kWh_Total"])

In [ ]:
temp_losses = (pv_model.all_models["PV_Thermal_Loss_kWh_Total"].sum() / pv_model.all_models["PV_Gen_kWh_Total"].sum() * 100)

print(f"{round(temp_losses, 3)}% change to power due to temperature losses")

In [ ]:
pv_model.models[0]["model_result"]

In [ ]:
pv_model.all_models

In [ ]:
pv_model.combined_model

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.daily

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.monthly

# ***MODEL ANALYSIS***